In [2]:
import os
import pandas as pd
import pyproj

In [21]:
# Open random points CSV as a Pandas DF
base_dir = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project"
csv_file_name = 'rand_suitability.csv'
csv_file_path = os.path.join(base_dir, csv_file_name)

# Read the CSV file into a DataFrame with specific settings
df = pd.read_csv(csv_file_path, header = 0, names=['X', 'Y', 'Score'])

# Display the DataFrame
print(df)


                X             Y  Score
0   462412.023682  4.981638e+06    4.0
1   460830.297119  5.001580e+06    4.5
2   476856.109131  4.963022e+06    4.0
3   481885.626282  4.977503e+06    4.0
4   466213.096680  5.006919e+06    4.5
5   483502.887329  4.971716e+06    4.0
6   467891.851685  4.971253e+06    4.0
7   444017.706482  4.983293e+06    4.0
8   454621.213318  4.987196e+06    4.5
9   447532.827087  4.976050e+06    4.0
10  472274.984924  4.988104e+06    4.0
11  455910.687927  4.995209e+06    4.5
12  475076.765503  4.980743e+06    4.0
13  477870.330872  4.994204e+06    4.0
14  472660.974915  4.972778e+06    5.0


In [22]:
# Reproject to GPS coordinates

# Define the input coordinate system (UTM Zone 15 in this example)
input_crs = pyproj.CRS.from_epsg(32615)

# Define the output coordinate system (WGS84 for GPS coordinates)
output_crs = pyproj.CRS.from_epsg(4326)

# Create a transformer to perform the coordinate conversion
transformer = pyproj.Transformer.from_crs(input_crs, output_crs, always_xy=True)

# Function to apply coordinate transformation to each row in the DataFrame
def transform_coordinates(row):
    x, y = row['X'], row['Y']
    lon, lat = transformer.transform(x, y)
    return pd.Series({'Latitude': lat, 'Longitude': lon})

# Apply the transformation function to each row in the DataFrame
df[['Latitude', 'Longitude']] = df.apply(transform_coordinates, axis=1)

# Display the transformed DataFrame
print(df)

                X             Y  Score   Latitude  Longitude
0   462412.023682  4.981638e+06    4.0  44.987188 -93.476806
1   460830.297119  5.001580e+06    4.5  45.166614 -93.498428
2   476856.109131  4.963022e+06    4.0  44.820229 -93.292733
3   481885.626282  4.977503e+06    4.0  44.950732 -93.229637
4   466213.096680  5.006919e+06    4.5  45.214946 -93.430297
5   483502.887329  4.971716e+06    4.0  44.898681 -93.208946
6   467891.851685  4.971253e+06    4.0  44.893980 -93.406636
7   444017.706482  4.983293e+06    4.0  45.000872 -93.710308
8   454621.213318  4.987196e+06    4.5  45.036761 -93.576130
9   447532.827087  4.976050e+06    4.0  44.935949 -93.664957
10  472274.984924  4.988104e+06    4.0  45.045850 -93.352053
11  455910.687927  4.995209e+06    4.5  45.108973 -93.560464
12  475076.765503  4.980743e+06    4.0  44.979692 -93.316112
13  477870.330872  4.994204e+06    4.0  45.100959 -93.281273
14  472660.974915  4.972778e+06    5.0  44.907900 -93.346320


In [23]:
# Create a new column formatted for UMN solar app GPS input
df['GPS'] = df['Latitude'].astype(str) + ', ' + df['Longitude'].astype(str)

print(df['GPS'])

0     44.987188229393915, -93.47680594016828
1      45.16661354685613, -93.49842765868615
2      44.82022885999228, -93.29273324429644
3      44.95073212981405, -93.22963650990752
4      45.21494613796155, -93.43029650569096
5      44.89868091665405, -93.20894566867919
6     44.893979709955936, -93.40663550132076
7      45.00087191691746, -93.71030806012966
8      45.03676122764181, -93.57612968442913
9      44.935948950733774, -93.6649573442522
10     45.04585031911932, -93.35205290177831
11    45.108972866276915, -93.56046387279397
12    44.979692228776116, -93.31611166827568
13      45.10095911419252, -93.2812732606209
14     44.90790028056747, -93.34632022845997
Name: GPS, dtype: object


In [43]:
# Adding True scores to the df

# List of scores from UMN Solar Tool
raw_umn_scores = [81, 76, 82, 75, 75, 78, 46, 61, 80, 90, 67, 61, 75, 21, 82]
umn_scores_1 = []

for loc in range(len(raw_umn_scores)):
    score = (0.04 * raw_umn_scores[loc]) + 1
    umn_scores_1.append(score)
df['True_Score'] = umn_scores_1

print(df['True_Score'])

0     4.24
1     4.04
2     4.28
3     4.00
4     4.00
5     4.12
6     2.84
7     3.44
8     4.20
9     4.60
10    3.68
11    3.44
12    4.00
13    1.84
14    4.28
Name: True_Score, dtype: float64


In [42]:
# Calculate accuracy of est score vs UMN Solar Tool score
df['Percentage_Error'] = abs((df['True_Score'] - df['Score']) / df['True_Score']) * 100

print(df['Percentage_Error'])

0       5.660377
1      11.386139
2       6.542056
3       0.000000
4      12.500000
5       2.912621
6      40.845070
7      16.279070
8       7.142857
9      13.043478
10      8.695652
11     30.813953
12      0.000000
13    117.391304
14     16.822430
Name: Percentage_Error, dtype: float64


In [48]:
df['Accuracy'] = df.apply(lambda row: row['Score'] / row['True_Score'] if row['Score'] < row['True_Score'] else row['True_Score'] / row['Score'], axis=1)

print(df['Accuracy'])

0     0.943396
1     0.897778
2     0.934579
3     1.000000
4     0.888889
5     0.970874
6     0.710000
7     0.860000
8     0.933333
9     0.869565
10    0.920000
11    0.764444
12    1.000000
13    0.460000
14    0.856000
Name: Accuracy, dtype: float64


In [49]:
# Output DataFrame to CSV file
csv_file_name = 'suitability_results.csv'
csv_file_path = os.path.join(base_dir, csv_file_name)
df.to_csv(csv_file_path, index = False)

print(f'DataFrame has been saved to {csv_file_path}')

DataFrame has been saved to C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\suitability_results.csv
